In [166]:
import numpy as np

class Tile:
    def __init__(self, id, image):
        self.id = id
        self.image = image
    
    def edges(self):
        return tuple(map(''.join, [
            self.image[0,:],
            self.image[:,-1],
            self.image[-1,::-1],
            self.image[::-1,0]
        ]))
    
    def flip(self):
        return Tile(self.id, self.image.transpose())
    
    def rotate(self):
        return Tile(self.id, np.rot90(self.image))
    
    def transformations(self):
        r = [self, self.flip()]
        t = self
        for _ in range(3):
            t = t.rotate()
            r.append(t)
            r.append(t.flip())
        return r
    
    def __eq__(lhs, rhs):
        return lhs.id == rhs.id
    
    def __hash__(self):
        return hash(self.id)

    def __str__(self):
        return f'Tile {self.id}:\n'+'\n'.join(map(''.join, self.image))
    
    def __repr__(self):
        return str(self)

def product(a, s=1):
    p = s
    for i in a:
        p *= i
    return p

tiles = []
with open('day20.txt') as d:
    r = d.read().strip()
    for t in r.split('\n\n'):
        lines = t.strip().split('\n')
        
        tiles.append(Tile(int(lines[0][5:-1]), np.array(list(map(list, lines[1:])))))
        

In [167]:
removed_edges = 0
for tile in tiles:
    for raw_edge in [
        tile.image[0,:],
        tile.image[:,-1],
        tile.image[-1,::-1],
        tile.image[::-1,0]
    ]:
        edge = ''.join(raw_edge)
        for other in tiles:
            if other is not tile:
                edges = other.edges()
                if edge in edges or edge[::-1] in edges:
                    break
        else:
            raw_edge[1:-1] = ['+']*(len(raw_edge)-2)
            removed_edges += 1
removed_edges

48

In [171]:
from math import sqrt
n = int(sqrt(len(tiles)))

grid = [[None]*n for _ in range(n)]

def fits(i, j, tile):
    edges = tile.edges()
    if i == 0 and edges[0][1] != '+':
        return False
    if i > 0 and grid[i-1][j].edges()[2][::-1] != edges[0]:
        return False
    if j == 0 and edges[3][1] != '+':
        return False
    if j > 0 and grid[i][j-1].edges()[1][::-1] != edges[3]:
        return False
    return True

available = set(tiles)
for i in range(n):
    for j in range(n):
        for tile in available:
            for transformation in tile.transformations():
                if fits(i, j, transformation):
                    grid[i][j] = transformation
                    break
            else:
                continue
            break
        available.remove(grid[i][j])
        
m = len(tiles[0].edges()[0])-2
image = np.array([[' ']*(n*m) for _ in range(n*m)])
for i in range(n):
    for j in range(n):
        image[i*m:(i+1)*m, j*m:(j+1)*m] = grid[i][j].image[1:-1,1:-1]
print('\n'.join(map(''.join, image)))

..#.............#.#...#............#.##..........#...#.......#..###.#.#..#....#.#....#..#.#.....
.........#..............#....#.........##.......#..#.#...#...#....#..#....#......#.##.....#.##..
.#...#......#..........#..#...........#..#...#...#.........#.#..............#...##.....##..##..#
............#.......#..#.....#..#......#.....#.#..#.#.####........#....##.#.#....##.#...........
....#.#..##...#.....#............................#.....###.##........#...........#.#...##...#...
..#.##.#..#.......................#...#....#..................#.#.......#..#.#...#.........#..##
....#.#..#...........#......#...#..#.#......#.........#.#..##.###..#.#.###........#.....#####..#
...##........#....##..##...#............#............#..##.##.#...##....#.........#.....##....#.
....##........#.#....#.....#......#...#......#.....##.......##..##.....#......#........##.....#.
.....#............#.............#...........##....##.....#.#...#..##............#..#...#....#...
...#....#.#.................#.

In [173]:
data = (image == '#')
seamonster = (np.array(list(map(list, """\
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.split('\n')))) == '#')

h, w = seamonster.shape

for t in Tile(0, data).transformations():
    img = t.image
    found = np.zeros(img.shape)
    seamonsters = 0
    for i in range(m*n - h):
        for j in range(m*n - w):
            if np.sum(np.logical_and(img[i:i+h, j:j+w], seamonster)) == np.sum(seamonster):
                found[i:i+h, j:j+w] = np.logical_or(found[i:i+h, j:j+w], seamonster)
                seamonsters += 1
    print(seamonsters, int(np.sum(img) - np.sum(found)))

0 1914
0 1914
0 1914
0 1914
0 1914
19 1629
0 1914
0 1914
